<a href="https://colab.research.google.com/github/adasegroup/ML2022_seminars/blob/master/seminar14/seminar_dim_reduction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Seminar: Dimensionality reduction

#### Seminar structure:

* SVD (Singular Value Decomposition) and PCA (Principal Component Analysis) from scratch
* Singular Value Decomposition of an Image.
* The ` Olivetti Faces dataset` component analysis.
* Instrinsic dimensionality estimation.
* Manifold learning.
* Autoencoding.

In [1]:
#linear algebra
import numpy as np
#data structures
import pandas as pd
#ml models
import scipy as sp
import sklearn
#plots
import matplotlib.pyplot as plt
%matplotlib inline
#beautiful plots
import seaborn as sns
import statsmodels.api as sm
#off the warnings
import warnings
warnings.filterwarnings("ignore")

## 1. SVD (Singular Value Decomposition) from the very scratch

The singular value decomposition of a matrix $A$ is the factorization of $A$ into the product of three matrices $A = U\Sigma V^T$ where the columns of $U$ and $V$ are orthonormal and the matrix $\Sigma$ is diagonal with positive real entries.

$$
X= U\Sigma V^T=
  \begin{matrix}
    \underbrace{\left[\begin{matrix} u_1 &  u_2 &  &  u_r\end{matrix}\right.}&
    \underbrace{\left.\begin{matrix} u_{r+1} & \dots &   u_m\end{matrix}\right]}\\
    col(X) & null(X^T)
  \end{matrix}
  \begin{bmatrix}
      \sigma_1 & 0 & \dots & 0 & 0 & \dots & 0 \\
         0 & \sigma_2  & \dots & 0 & 0 & \dots & 0 \\
         \dots& & & & &  \\
         0 & 0 & \dots & \sigma_r  & 0 & \dots & 0 \\
         0 & 0 & \dots & 0 & 0 & \dots & 0 \\
         \dots& & & & &  \\
         0 & 0 & \dots & 0 & 0 & \dots & 0
  \end{bmatrix}
  \begin{bmatrix}
     v_1^T \\  v_2^T \\ \dots \\  v_r^T \\
     v_{r+1}^T \\ \dots \\  v_n^T
  \end{bmatrix}
  \begin{matrix}
    \left.\vphantom{\begin{bmatrix}
       \ v_1^T \\  v_2^T \\ \dots \\ \vect v_r^T
       \end{bmatrix}}\right\} row(X) \\
    \left.\vphantom{\begin{bmatrix}
      \ v_{r+1}^T \\ \dots \\  v_n^T
    \end{bmatrix}}\right\} null(X)
  \end{matrix}
$$

<img src="https://www.askpython.com/wp-content/uploads/2020/11/SVD-1.jpg" alt="Drawing" style="width: 300px;" />

Credit for: https://www.askpython.com/wp-content/uploads/2020/11/SVD-1.jpg

#### Questions:

1. Is `SVD()` use iterative search or there is some closed solution?
2. What is the criterion of good decomposition?
3. What is the difference of `PCA` and `SVD`?


To refresh your math - you can check lectures or here are some useful resources:

[1] [Machine Learning — Singular Value Decomposition (SVD) & Principal Component Analysis (PCA)](https://jonathan-hui.medium.com/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491#:~:text=What%20is%20the%20difference%20between,PCA%20skips%20less%20significant%20components.)

[2] Dan Margalit, Joseph Rabinoff. Interactive Linear Algebra ([link](https://textbooks.math.gatech.edu/ila/index2.html))

[3] Strang, Gilbert. Linear Algebra and Its Applications

#### Power Method:

In order to implement SVD, we will need *power method*. Recap of power method to compute largest eigenvalue $\lambda$ and corresponding eigenvector $v$ (for $k$-th step):
1. $x_{k+1} \leftarrow A x_k$
2. $x_{k+1} \leftarrow \dfrac{x_{k+1}}{ \| x_{k+1} \|}$

In that case $x_{k+1} \rightarrow v$ and $A x_{k+1} x^T_{k+1} \rightarrow \lambda$ with increasing $k$.




#### How can it help to find SVD?

If $A = U \Sigma V^T$ then $A^TA = V \Sigma^T U^T U \Sigma V^T = V \Sigma^2 V^T$ which is a diagonalization of a matrix $A^TA$. Then $\Sigma^2_{ii} = \sigma^2_i$ with corresponding eigenvector in first column of $V$.

In [2]:
class MySVD:
    """
    CLass for iterative SVD search
    """
    def __init__(self, tol=1e-10, eigval_thr=1e-10, max_iter=100, random_state=0, verbose=False):
        """
        Arguements:
        - tol: tolerance of difference
        - eigval_thr: theshold for eigvalues

        """
        self.tol = tol
        self.eigval_thr = eigval_thr
        self.max_iter = max_iter
        self.gen = np.random.RandomState(random_state)
        self.verbose = verbose

    def _initialize(self, X, n_components):
        self.X = np.array(X)
        assert self.X.ndim == 2
        # singular values
        self.eig_vals = []
        # left singular vectors
        self.eig_vecs_left = []
        # right singular vectors
        self.eig_vecs_right = []

        n_components = self.X.shape[0] if n_components is None else n_components
        self.n_components = min(self.X.shape[0], self.X.shape[1], n_components)

    def _make_step(self, X, a, b):


        ### YOUR CODE HERE ### # make one step of power method


        return new_a, new_b

    def __call__(self, X, n_components=None):
        self._initialize(X, n_components)
        # iteretively search for the components
        for n_component in range(self.n_components):
            a = self.gen.rand(self.X.shape[0])
            b = self.gen.rand(self.X.shape[1])

            prev_F = np.sum((self.X - a[:, None] * b[None, :])**2)
            delta_F = np.inf

            # stop criterion
            for n_iter in range(self.max_iter):
                a, b = self._make_step(self.X, a, b)
                F = np.sum((self.X - a[:, None] * b[None, :])**2)
                delta = prev_F - F
                prev_F = F
                self._print('n_eig = {}, n_iter = {}: delta = {:.5f}, F = {:.5f}.'.format(
                        len(self.eig_vals), n_iter, delta, F))
                if np.abs(delta) <= self.tol:
                    self._print('Frobenius norm equals {}. Stopping iterations for n_eig={}.'.format(
                        F, len(self.eig_vals)))
                    break

            eig_val = np.sum(a**2) * np.sum(b**2) # eigen values
            # stop criterion
            if eig_val < self.eigval_thr:
                self._print('Singular value {} equals {}. Stopping iterations.'.format(
                    len(self.eig_vals) + 1, eig_val))
                break

            self.eig_vals.append(eig_val)
            self.eig_vecs_left.append(a[:, None] / np.sqrt(np.sum(a**2)))
            self.eig_vecs_right.append(b[:, None] / np.sqrt(np.sum(b**2)))
            self.X -= a[:, None] * b[None, :]

        self.U = np.hstack(self.eig_vecs_left)
        self.V = np.hstack(self.eig_vecs_right)
        self.D = np.sqrt(self.eig_vals)
        return self.U, self.D, self.V.T

    def _print(self, msg):
        if self.verbose:
            print(msg)

In [ ]:
test_array = np.random.randn(20, 5)
u_np, d_np, v_np =  ### YOUR CODE HERE ### # compare with np.linalg
u, d, v = MySVD(verbose='True')(test_array)

assert np.allclose(d, d_np, atol=1e-2)
assert np.allclose(np.dot(np.dot(u, np.diag(d)), v), test_array, atol=1e-2)

In [4]:
class MyPCA:
    def __init__(self, n_components=10, **svd_kwargs):
        self.n_components=n_components
        self.svd = MySVD(**svd_kwargs)

    def fit(self, X):
        X = np.array(X)
        self.n_samples, self.n_features = X.shape

        # centering
        self.mean_ = X.mean(axis=0)
        X = X - self.mean_[None, :]


        # full variance - as a sum of distances to the center
        self.full_variance_ = np.sum(np.mean(X**2, axis=0))

        # SVD
        self.U, self.D, self.V = ### YOUR CODE HERE ### # compute svd with n_components

        # eigen vectors X^TX in V.T
        self.n_components_ = len(self.D)
        self.components_ = self.V

        # the portion of variance explained
        self.explained_variance_ = self.D**2 / self.n_samples
        self.explained_variance_ratio_ = self.explained_variance_ / self.full_variance_

    def transform(self, X):
        return  ### YOUR CODE HERE ###

    def fit_transform(self, X):
        self.fit(X)
        return self.transform(X)

    def recover(self, X):
        return  ### YOUR CODE HERE ###

In [ ]:
from sklearn.linear_model import LinearRegression

# toy data
plt.figure(figsize=(9, 6))
np.random.seed(2)
X = np.random.randn(200)
Y = X + np.random.randn(len(X)) * 0.5
model = LinearRegression()
model.fit(X.reshape(-1, 1), Y)
test_x = np.linspace(-4, 4, 100)
predictions = model.predict(test_x.reshape(-1, 1))
plt.plot(X, Y, 'o')
plt.plot(test_x, predictions, label='No Outliers', color='C0')

# add noise
sub_inds = np.random.choice(len(X), 10, replace=False)
XA = X.copy()
YA = Y.copy()
YA[sub_inds] = Y[sub_inds] + np.random.rand(len(sub_inds)) * 10
model.fit(XA.reshape(-1, 1), YA)
predictions = model.predict(test_x.reshape(-1, 1))
plt.plot(test_x, predictions, label='With Outliers')
plt.plot(XA[sub_inds], YA[sub_inds], 'o', color='C1')
plt.legend(loc='best')

plt.xlabel('x')
plt.ylabel('y')

In [ ]:
X = np.stack([X,Y]).T
#X = np.stack([XA,YA]).T

pca =  ### YOUR CODE HERE ###
pca.fit(X)

print("Explained variance: {}".format(pca.explained_variance_ratio_))
print("Principal components:")
print("First: {}".format(pca.components_[0]))
print("Second: {}".format(pca.components_[0]))


plt.figure(figsize=(9, 6))

plt.scatter(X[:, 0], X[:, 1], s=25, c='r', marker='o', edgecolor='k')

for var_ratio, direction in zip(pca.explained_variance_ratio_, pca.components_):
    d = 7 * np.sqrt(var_ratio) * direction
    plt.plot([0, d[0]], [0, d[1]], '-k', lw=2)

plt.axis('equal')

##### Watch in the next series (seminar): "Anomaly detection with PCA"

## 2. Singular Value Decomposition and PCA of an Image for data compression

In [ ]:
# library for images
from PIL import Image

img = Image.open('fruits.png')
imggray = img.convert('LA')
plt.figure(figsize=(9, 6));
plt.imshow(img);

In [ ]:
imgmat = np.array(list(imggray.getdata(band=0)), float)
imgmat.shape = (imggray.size[1], imggray.size[0])
imgmat = np.array(imgmat)
plt.figure(figsize=(9,6))
plt.imshow(imgmat, cmap='gray');

In [9]:
# you can try this one, yet it converges much longer

# U, sigma, V = MySVD(verbose=False)(imgmat)
U, sigma, V = np.linalg.svd(imgmat, full_matrices=False)


Computing an approximation of the image using the first column of  $U$  and first row of  $V$  reproduces the most prominent feature of the image, the light area on top and the dark area on the bottom. The darkness of the arch causes the extra darkness in the middle of the reconstruction. Each column of pixels in this image is a different weighting of the same values,  $u_1$ :

In [ ]:
from sklearn.decomposition import PCA
### PCA
pca = ### YOUR CODE HERE ###
reconstruction = ### YOUR CODE HERE ###

### SVD
reconstruction = ### YOUR CODE HERE ###
plt.imshow(reconstruction, cmap='gray');

If we will be able to see the appearance from 2-nd component?

In [ ]:
# for PCA
for i in range(2, 4):
    pca =  ### YOUR CODE HERE ###
    reconstruction = ### YOUR CODE HERE ###
    plt.imshow(reconstruction, cmap='gray')
    title = "n = %s" % i
    plt.title(title)
    plt.show()

### Ok, how many do we need?

In [ ]:
# ok how many do we need
for i in range(0, 51, 5):
    # for PCA
    pca = ### YOUR CODE HERE ###
    reconstruction =  ### YOUR CODE HERE ###
    plt.imshow(reconstruction, cmap='gray')
    title = "n = %s" % i
    plt.title(title)
    plt.show()

**For my perspective, `[:45]` reconstruction is good enough. Thus we can store less data.**

Questions:
* What portion of the data we can discard this way?
* Can we estimate the intrinsic dimension of the image through PCA?

In [ ]:
from sklearn.decomposition import PCA
pca = PCA().fit(imgmat)

plt.figure(figsize=(14, 4))

plt.step(### YOUR CODE HERE ###, label='cumulative explained variance')
plt.title('Cumulative explained variance', fontsize=16)
plt.xlabel('# principal components', fontsize=12)
plt.ylabel('Cumulative explained variance', fontsize=12)

plt.hlines(y=0.95, xmin=0, xmax= len(pca.components_), colors='r', linestyles='dashed', label='95% explained variance')
plt.legend(loc='lower right')
plt.show()

## 3. The `Olivetti Faces dataset` component analysis.

In [ ]:
import numpy as np
from sklearn.datasets import fetch_olivetti_faces
from numpy.random import RandomState


data = fetch_olivetti_faces(shuffle=True, random_state= 42 ).data
target = fetch_olivetti_faces(shuffle=True, random_state= 42).target

data.shape

Let's look at the dataset

In [ ]:
plt.figure(figsize=(12, 5))
for i in range(40):
    plt.subplot(4, 10, i + 1)
    plt.imshow(data[i].reshape((64, 64)), cmap='gray')
    plt.axis('off')

Let's look at the importance of the main components

In [ ]:
from sklearn.decomposition import PCA
pca = PCA().fit(data)

plt.figure(figsize=(14, 4))

plt.step(range(len(pca.components_)), np.cumsum(pca.explained_variance_ratio_), label='cumulative explained variance')
plt.title('Cumulative explained variance', fontsize=16)
plt.xlabel('# principle components', fontsize=12)
plt.ylabel('Cumulative explained variance', fontsize=12)

plt.hlines(y=0.95, xmin=0, xmax= len(pca.components_), colors='r', linestyles='dashed', label='95% explained variance')
plt.legend(loc='lower right')
plt.show()

Let's look at the principial components

In [ ]:
plt.figure(figsize=(1, 1))
plt.imshow(pca.mean_.reshape((64, 64)), cmap='gray')
plt.axis('off')
plt.show()

plt.figure(figsize=(12, 6))
for i in range(50):
    plt.subplot(5, 10, i + 1)
    plt.imshow(pca.components_[i].reshape((64, 64)), cmap='gray')
    plt.axis('off')

Let's see how does the faces are reconstructed by principial components. For the sake of visibility, we take 10 components, yet is is not enough for good reconstruction:(

In [ ]:
pca = PCA(n_components=10)
transform_X = pca.fit_transform(data)

for i in range(10):
    X_hat = pca.inverse_transform(transform_X[i])
    plt.figure(figsize=(16, 1))

    plt.subplot(1, 13, 1)
    plt.imshow(data[i].reshape((64, 64)), cmap='gray')
    plt.axis('off')
    plt.title('face')

    plt.subplot(1, 13, 2)
    plt.imshow(X_hat.reshape((64, 64)), cmap='gray')
    plt.axis('off')
    plt.title('recon.')

    plt.subplot(1, 13, 3)
    plt.imshow(pca.mean_.reshape((64, 64)), cmap='gray')
    plt.axis('off')
    plt.title('= 1')

    for j in range(10):
        plt.subplot(1, 13, 4 + j)
        plt.imshow(pca.components_[j].reshape((64, 64)), cmap='gray')
        plt.axis('off')
        plt.title('{}{:.1f}'.format('+' if transform_X[i, j] >= 0 else '',
                                    transform_X[i, j]))

    plt.show()

2D visualization via PCA

In [34]:
from matplotlib import offsetbox
from matplotlib.offsetbox import AnnotationBbox, OffsetImage

def plot_embedding(X, y, images_small=None, title=None):
    """
    Nice plot on first two components of embedding with Offsets.

    """
    # take only first two columns
    X = X[:, :2]
    # scaling
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)
    plt.figure(figsize=(13,8))
    ax = plt.subplot(111)

    for i in range(X.shape[0] - 1):
        plt.text(X[i, 0], X[i, 1], str(y[i]),
                 color=plt.cm.RdGy(y[i]),
                 fontdict={'weight': 'bold', 'size': 12})
        if images_small is not None:
            imagebox = OffsetImage(images_small[i], zoom=.4, cmap = 'gray')
            ab = AnnotationBbox(imagebox, (X[i, 0], X[i, 1]),
                xycoords='data')
            ax.add_artist(ab)

    if hasattr(offsetbox, 'AnnotationBbox'):
        # only print thumbnails with matplotlib > 1.0
        shown_images = np.array([[1., 1.]])
        for i in range(X.shape[0]):
            dist = np.sum((X[i] - shown_images) ** 2, 1)
            if np.min(dist) < 4e-1:
                # don't show points that are too close
                continue
    if title is not None:
        plt.title(title)

In [ ]:
X_projected = ### YOUR CODE HERE ###
data_pic = data.reshape((-1, 64, 64))
plot_embedding(X_projected, target, data_pic, "PCA decomposition, projection on first two components  ")

## 4. Estimation of intrinsic dimension

From the paper:


*There is a consensus in the high-dimensional data analysis community that the only reason any methods work in very high dimensions is that, in fact, the data are not truly high-dimensional. Rather, they are embedded in a high-dimensional space,
but can be efficiently summarized in a space of a much lower dimension, such as a nonlinear manifold.*

*Then one can reduce dimension without losing much information for many types of real-life high-dimensional data, such as images, and avoid many of the “curses of dimensionality”. Learning these data manifolds can improve
performance in classification and other applications, but if the data structure is
complex and nonlinear, dimensionality reduction can be a hard problem.*

#####  NIPS 2004: https://papers.nips.cc/paper/2577-maximum-likelihood-estimation-of-intrinsic-dimension.pdf

### Idea of the method

The idea of the nearest-neighbor intrinsic dimension estimation is based on fact of the  that the number of sample points in $\mathbf{X} = \mathbb{R}^n$ falling into a ball around $\mathbf{x}$ is proportional to the radius of of the ball $R$, the dimension of the ball $d$ and the density $f(x)$ for which the well-known estimator exists

$$f(x) = \frac{k/n}{R^d V(d)},$$

where $V(d) = \pi^{d/2} [\Gamma (d/2+1)]^{-1}$ is the volume of unit sphere in $\mathbb{R}^d$ and $\Gamma(d) = (d - 1)!$ is the Gamma function.

Then the intrinsic dimension $d$ of the sample can be regressed out or estimated by the maximum likelihood appoarch.

In [21]:
"""
Implementation of 'Maximum Likelihood Estimation of Intrinsic Dimension' by Elizaveta Levina and Peter J. Bickel

how to use
----------

The goal is to estimate intrinsic dimensionality of data, the estimation of dimensionality is scale dependent
(depending on how much you zoom into the data distribution you can find different dimesionality), so they
propose to average it over different scales, the interval of the scales [k1, k2] are the only parameters of the algorithm.

This code also provides a way to repeat the estimation with bootstrapping to estimate uncertainty.

Here is one example with swiss roll :

from sklearn.datasets import make_swiss_roll
X, _ = make_swiss_roll(1000)

k1 = 10 # start of interval(included)
k2 = 20 # end of interval(included)
intdim_k_repeated = repeated(intrinsic_dim_scale_interval,
                             X,
                             mode='bootstrap',
                             nb_iter=500, # nb_iter for bootstrapping
                             verbose=1,
                             k1=k1, k2=k2)
intdim_k_repeated = np.array(intdim_k_repeated)
# the shape of intdim_k_repeated is (nb_iter, size_of_interval) where
# nb_iter is number of bootstrap iterations (here 500) and size_of_interval
# is (k2 - k1 + 1).

"""
# from tqdm import tqdm
import pandas as pd
import numpy as np
from sklearn.neighbors import NearestNeighbors

def intrinsic_dim_sample_wise(X, k=5):
    neighb = NearestNeighbors(n_neighbors=k+1).fit(X)
    dist, ind = neighb.kneighbors(X) # distances between the samples and points
    dist = dist[:, 1:] # the distance between the first points to first points (as basis ) equals zero
    # the first non trivial point
    dist = dist[:, 0:k]# including points k-1
    assert dist.shape == (X.shape[0], k) # requirments are there is no equal points
    assert np.all(dist > 0)
    d = np.log(dist[:, k - 1: k] / dist[:, 0:k-1]) # dinstance betveen the bayeasan statistics
    d = d.sum(axis=1) / (k - 2)
    d = 1. / d
    intdim_sample = d
    return intdim_sample

def intrinsic_dim_scale_interval(X, k1=10, k2=20):
    X = pd.DataFrame(X).drop_duplicates().values # remove duplicates in case you use bootstrapping
    intdim_k = []
    for k in range(k1, k2 + 1): # in order to reduse the noise by eliminating of the nearest neibours
        m = intrinsic_dim_sample_wise(X, k).mean()
        intdim_k.append(m)
    return intdim_k

def repeated(func, X, nb_iter=100, random_state=None, mode='bootstrap', **func_kw):
    if random_state is None:
        rng = np.random
    else:
        rng = np.random.RandomState(random_state)
    nb_examples = X.shape[0]
    results = []

    iters = range(nb_iter)
    for i in iters:
        if mode == 'bootstrap':# and each point we want to resample with repeating points to reduse the errors
            #232 111 133
            Xr = X[rng.randint(0, nb_examples, size=nb_examples)]
        elif mode == 'shuffle':
            ind = np.arange(nb_examples)
            rng.shuffle(ind)
            Xr = X[ind]
        elif mode == 'same':
            Xr = X
        else:
            raise ValueError('unknown mode : {}'.format(mode))
        results.append(func(Xr, **func_kw))
    return results

In [ ]:
import warnings
warnings.filterwarnings("ignore")

k1 = 1 # start of interval(included)
k2 = 5 # end of interval(included)
nb_iter = 3 # more iterations more accuracy
# intrinsic_dim_scale_interval gives better estimation
intdim_k_repeated = repeated(intrinsic_dim_scale_interval,


                             ### YOUR CODE HERE ###, # nb_iter for bootstrapping


                             )
intdim_k_repeated = np.array(intdim_k_repeated)

x = np.arange(k1, k2+1)

plt.figure(figsize=(14, 4))
plt.plot(x, np.mean(intdim_k_repeated, axis=0), 'b', label='Mean') # it is the mean walue
plt.fill_between(x,
                 np.mean(intdim_k_repeated, axis=0) -
                 2 * np.std(intdim_k_repeated, axis=0),
                 np.mean(intdim_k_repeated, axis=0) +
                 2 * np.std(intdim_k_repeated, axis=0),
                 alpha=0.3,
                 label='CI=95%',
                 color='g')
plt.xlabel("Nearest Neigbours")
plt.ylabel("Intrinsic Dimensionality")
plt.legend()
plt.show()

For more methods of intrinsic dimensionality estimation we may refer to ```scikit-dimension``` package by this ([link](https://scikit-dimension.readthedocs.io/en/latest/index.html)). It contains different implementations of many methods. Here is the example of estimation using MLE:



In [23]:
# !pip install scikit-dimension

In [ ]:
import skdim
int_D = skdim.id.MLE().fit_transform(data)
print(int_D)

## 5. Manifold learning  

It is big part of research on the application of geometry and in particular differential geometry to machine learning. If you are interested - next term it will be course by **Prof. Bernstein** - you are welcomed.

We are going to touch a couple popular algorithms from: https://scikit-learn.org/stable/modules/manifold.html

The manifold learning methods also assumes non-linear algorithms for dimensionality reduction.

Questions:
* Is `PCA` linear?
* Can we compose non-linear `PCA` from the linear?

### 5.1 Independent Component Analysis

**Lets go with some other liner method : Independent Component Analysis `ICA`.**
FastICA algorithm: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.FastICA.html

The problem of blind signal separation.

$$X = AS$$

where A – mixing matrix, S – source of the signals. The assumptions are:
- The source signals are statistically independent of each other.
- The values in each source signal have non-Gaussian distributions (at most one).

<img src="https://cf2.ppt-online.org/files2/slide/c/CarDgVY6t91bypGImvqBcR3OdAjWhFTZ2JQxL4ofM/slide-6.jpg" alt="Drawing" style="width: 700px;" />
Credit for: https://cf2.ppt-online.org/files2/slide/c/CarDgVY6t91bypGImvqBcR3OdAjWhFTZ2JQxL4ofM/slide-6.jpg

In [25]:
# let's leave 20 people from faces to get more comprehencible visualisation

data = fetch_olivetti_faces(shuffle=True, random_state= 42 ).data
target = fetch_olivetti_faces(shuffle=True, random_state= 42).target


data = data[target <20]
target = target[target <20]

In [ ]:
from sklearn.decomposition import FastICA

X_projected = FastICA(20, random_state = 42).fit_transform(data)
data_pic = data.reshape((-1, 64, 64))
plot_embedding(X_projected, target, data_pic, "ICA decomposition, projection on two components")

**The most popular nonlinear algorithms: tSNE,  MDS, Isomap**

t-SNE converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data. t-SNE has a cost function that is not convex, i.e. with different initializations we can get different results.

1. For each data point $x_i$ we’ll center a Gaussian distribution over that point. Then we measure the density of all points $x_j$ under that Gaussian distribution. Then renormalize for all points. This gives us a set of probabilities $P_{ij}$ for all points in higher dimensional space.

2. Get the second set of probablities $Q_{ij}$ for Cauchy distribution (with is Students t-distribution with one degree of freedom), which allow for better modeling of far apart distances, because of heavier tails.

3. Map these two sets of probabilities ($ P_{ij}; Q_{ij}$)to each other, optimizing KL-divergence.

Where Kullback–Leibler divergence can be defined as

$$ D_{KL} (P || Q) = \sum P(x) log(\frac{P(x)}{Q(x)})$$

In [ ]:
from sklearn.manifold import TSNE


tsne = TSNE(n_components=2, n_iter = 1000, metric='euclidean', learning_rate= 10, verbose=2, random_state = 42)
X_projected = tsne.fit_transform(data)
data_pic = data.reshape((-1, 64, 64))
plot_embedding(X_projected, target, data_pic, "tSNE decomposition, projection on two components")

The effect of the perplexity

In [28]:
from sklearn.datasets import make_blobs

In [29]:
X, y = make_blobs(n_samples=500, n_features=10, centers=5)

In [ ]:
tsne = TSNE(n_components=2)
X_hat = tsne.fit_transform(X)

plt.figure(figsize=(10, 6))
plt.scatter(X_hat[:, 0], X_hat[:, 1], c=y, cmap='Set1')
plt.show()

Let's see how does the displacement of points in the 2D space changes with respect to the `perplexity` parameter (smoothed effective number of the neighbours)

In [ ]:
for perplexity in [1, 5, 15, 30, 50, 75, 100]:

    tsne = TSNE(n_components=2, perplexity=perplexity)
    X_hat = tsne.fit_transform(X)

    plt.figure(figsize=(10, 6))
    plt.scatter(X_hat[:, 0], X_hat[:, 1], c=y, cmap='Set1')
    plt.title('Perp = {}'.format(perplexity))
    plt.show()

### Multi-dimensional Scaling (MDS)

Multidimensional scaling (MDS) seeks a low-dimensional representation of the data in which the distances respect well the distances in the original high-dimensional space.

*Modern Multidimensional Scaling - Theory and Applications, Borg, I.; Groenen P. Springer Series in Statistics (1997)*

In [ ]:
from sklearn.manifold import MDS

X_projected = MDS(n_components=2).fit_transform(data)
data_pic = data.reshape((-1, 64, 64))
plot_embedding(X_projected, target, data_pic, "MDS decomposition, projection on two components")

### Isometric Mapping (Isomap)

Isomap can be viewed as an extension of Multi-dimensional Scaling (MDS) or Kernel PCA. Isomap seeks a lower-dimensional embedding which maintains geodesic distances between all points. Goes in three stages:

1. **Nearest neighbor search**.

2. **Shortest-path graph search.**

3. **MDS with respect to the geodesic distances**.

In [ ]:
from sklearn.manifold import Isomap

X_projected = Isomap(n_components=2).fit_transform(data)
data_pic = data.reshape((-1, 64, 64))
plot_embedding(X_projected, target, data_pic, "Isomap decomposition, projection on  two components")